In [ ]:
# As you will recall, `numpy.exp` works on the CPU, but, cannot be used in GPU implmentations.
# This import will work for the CPU-only boilerplate code provided below, but
# you will need to modify this import before your GPU implementation will work.
"""
from numpy import exp
"""

# Solution:
from math import exp 

In [ ]:
# Modify these 3 function calls to run on the GPU.
"""
def normalize(grayscales):
    return grayscales / 255

def weigh(values, weights):
    return values * weights
        
def activate(values):
    return ( exp(values) - exp(-values) ) / ( exp(values) + exp(-values) )
"""
# Solution:

@vectorize(['float32(float32)'], target = 'cuda')
def normalize(grayscales):
    return grayscales / 255

@vectorize(['float32(float32, float32)'], target = 'cuda')
def weigh(values, weights):
    return values * weights
        
@vectorize(['float32(float32)'], target = 'cuda')
def activate(values):
    return ( exp(values) - exp(-values) ) / ( exp(values) + exp(-values) )

In [ ]:
# Modify the body of this function to optimize data transfers and therefore speed up performance.
# As a constraint, even after you move work to the GPU, make this function return a host array.
def create_hidden_layer(n, greyscales, weights, exp, normalize, weigh, activate):
    
    greyscales_device = cuda.to_device(greyscales)
    weights_device = cuda.to_device(weights)
    
    normalized_out_device = cuda.device_array(shape=(n,), dtype=np.float32)  # does not initialize the contents, like np.empty()
    weighted_out_device = cuda.device_array(shape=(n,), dtype=np.float32)  # does not initialize the contents, like np.empty()
    activated_out_device = cuda.device_array(shape=(n,), dtype=np.float32)  # does not initialize the contents, like np.empty()

    normalize(greyscales_device, out = normalized_out_device)
    weigh(normalized_out_device, weights_device, out = weighted_out_device)
    activate(weighted_out_device, out = activated_out_device)
    
    # The assessment mechanism will expect `activated` to be a host array, so,
    # even after you refactor this code to run on the GPU, make sure to explicitly copy
    # `activated` back to the host.
    return activated_out_device.copy_to_host()